In [86]:
import pandas as pd
import numpy as np

In [87]:
universes = pd.read_csv('./Data/univ_h.csv')
universes = universes.T
universes.columns = universes.iloc[0]
universes = universes.drop(universes.index[0])

In [88]:
adj_close = pd.read_csv('./Data/adjusted.csv', header=None)
adj_close.iloc[0, :] = [''.join(index.split()) for index in adj_close.iloc[0, :]]
adj_close.columns = adj_close.iloc[0, :]
adj_close = adj_close.drop(0)
adj_close['Date'] = pd.to_datetime(adj_close['Date'], format='%Y%m%d')

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,2

In [89]:
df_price = pd.DataFrame()

for year in universes.columns:
    df_year = adj_close[adj_close['Date'].dt.year == year].copy()
    ticker = universes[year].replace(0, np.nan).dropna().index
    df_year = df_year.loc[:, ['Date'] + list(ticker)]
    df_year.index = df_year['Date']
    df_year.drop('Date', axis=1, inplace=True)

    df_price = pd.concat([df_price, df_year])

df_price = df_price.apply(lambda x: pd.to_numeric(x))
df_price.fillna(0, inplace=True)

In [90]:
daily_returns = df_price.apply(lambda x: np.log(x / x.shift(1))).fillna(0)
volatility = daily_returns.rolling(window=21).std()
volatility = volatility.applymap(lambda x: 0.005 if x < 0.005 else x)

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [91]:
five_days_returns = df_price.apply(lambda x: np.log(x / x.shift(5))).fillna(0)

In [92]:
std_returns =  (five_days_returns / volatility).fillna(0)

In [93]:
idt_code = pd.read_csv('./Data/tickers.csv', index_col=0, header=None)
idt_code.columns = ['GIC_Code']
idt_code['GIC_Industry'] = idt_code['GIC_Code'].astype(str).str[:6]

In [94]:
# Merge the industry code with the returns DataFrame
std_returns_with_industry = std_returns.T.join(idt_code['GIC_Industry'])

industry_returns = std_returns_with_industry.groupby('GIC_Industry').mean().T

std_returns_with_industry = std_returns_with_industry.T.apply(
    lambda column: column[:-1] - industry_returns.loc[:, column[-1]] if column[-1] in industry_returns.columns else column[:-1]
)

In [95]:
d_returns = daily_returns.T.join(idt_code['GIC_Industry'])

industry_d_returns = d_returns.groupby('GIC_Industry').mean().T

d_returns_with_industry = d_returns.T.apply(
    lambda column: column[:-1] - industry_d_returns.loc[:, column[-1]] if column[-1] in industry_d_returns.columns else column[:-1]
)

In [96]:
import statsmodels.api as sm

t1_returns = d_returns_with_industry.shift(-1).fillna(0)

reg_coefficient = pd.DataFrame(index=std_returns_with_industry.index, columns=['beta', 'Rsquared'])

for i in range(len(std_returns_with_industry)):
    X = std_returns_with_industry.iloc[i, :]
    y = t1_returns.iloc[i, :]

    X = pd.to_numeric(X, errors='coerce')
    y = pd.to_numeric(y, errors='coerce')

    # X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()

    # print(model.summary())

    beta = model.params[0]
    r_squared = model.rsquared

    reg_coefficient.iloc[i, 0] = beta
    reg_coefficient.iloc[i, 1] = r_squared

reg_coefficient = reg_coefficient.loc['20050103':, :].dropna()
reg_coefficient

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1631: RuntimeWarning: invalid value encountered in subtract
  return self.model.wendog - self.model.predict(
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


,beta,Rsquared
2005-01-03,-0.000588,0.002017
2005-01-04,0.001365,0.029069
2005-01-05,-0.000746,0.015589
2005-01-06,0.000296,0.00407
2005-01-07,-0.000504,0.017454
...,...,...
2024-12-23,-0.00151,0.141757
2024-12-24,0.000054,0.000146
2024-12-26,-0.001425,0.105594
2024-12-27,-0.001663,0.05086
